In [11]:
import os
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Callable, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec
from scipy.optimize import curve_fit
from scipy.optimize import minimize

In [12]:
# def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
#     """Compute coefficient of determination."""
#     ss_res = np.sum((y_true - y_pred) ** 2)
#     ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
#     return 1 - ss_res / ss_tot


def calculate_r2(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Compute coefficient of determination."""

    if y_true.shape != y_pred.shape:
        raise ValueError("y_true and y_pred must have the same shape.")

    if y_true.size < 2:  # R^2 не определен для менее чем 2 точек
        return np.nan  # Или можно поднять ошибку

    y_mean = np.mean(y_true)

    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - y_mean) ** 2)

    # Обработка случая, когда ss_tot равно нулю (все y_true одинаковы)
    if ss_tot == 0:
        return (
            1.0 if ss_res == 0 else 0.0
        )  # Если все точки одинаковы и модель их точно предсказывает, R^2 = 1

    return 1 - (ss_res / ss_tot)


@dataclass
class ModelSpec:
    name: str
    t1: str
    t2: str
    func: Callable
    init_guess: List[float]
    bounds: List


MODELS = {
    0: {
        "name": "linear",
        "t1": "c0 * x + c1",
        "t2": "{0} * x + {1}",
        "func": lambda x, c0, c1: c0 * x + c1,
        "init_guess": [0.5, 0.5],
        "bounds": ((-np.inf, -np.inf), (np.inf, np.inf)),
    },
    1: {
        "name": "exp001",
        "t1": "c0 * EXP[c1 * x] + c2",
        "t2": "{0} * EXP[{1} * x] + {2}",
        "func": lambda x, c0, c1, c2: c0 * np.exp(c1 * x) + c2,
        "init_guess": [0.5, 0.5, 0.5],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
    2: {
        "name": "exp002",
        "t1": "c0 * EXP[c1 * x] + c2 * EXP[c3 * x] + c4",
        "t2": "{0} * EXP[{1} * x] + {2} * EXP[{3} * x] + {4}",
        "func": lambda x, c0, c1, c2, c3, c4: c0 * np.exp(c1 * x)
        + c2 * np.exp(c3 * x)
        + c4,
        "init_guess": [0.5, 0.5, 0.5, 0.5, 0.5],
        "bounds": (
            (-np.inf, -np.inf, -np.inf, -np.inf, -np.inf),
            (np.inf, np.inf, np.inf, np.inf, np.inf),
        ),
    },
    3: {
        "name": "log001",
        "t1": "c1 * LN[x + c2] + c3",
        "t2": "{0} * LN[x + {1}] + {2}",
        "func": lambda x, c1, c2, c3: c1 * np.log(x + c2) + c3,
        "init_guess": [0.5, 0.5, 0.5],
        "bounds": ((-np.inf, 1e-10, -np.inf), (np.inf, np.inf, np.inf)),
    },
    4: {
        "name": "pow001",
        "t1": "c1 * (x ^ c2) + c3",
        "t2": "{0} * (x ^ {1}) + {2}",
        "func": lambda x, c1, c2, c3: c1 * (x**c2) + c3,
        "init_guess": [0.5, 0.5, 0.5],
        "bounds": ((-np.inf, -np.inf, -np.inf), (np.inf, np.inf, np.inf)),
    },
}

In [13]:
data = {}

for ID, PATH in enumerate(
    [
        r"1753634027976959_Fig9_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.05\sim_history.txt",
        r"1753636833312678_Fig10_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.17_Pp1.0\sim_history.txt",
        # r"1753790596710720_Fig10_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.085_Pp1.0\sim_history.txt",
        # r"1753791383930285_Fig10_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.0425_Pp1.0\sim_history.txt",
        # r"1753790448886155_Fig10_X50Y200Z50_T3e2_C9.58767e-8_Nt1e10_Pb0.17_Pp2.0\sim_history.txt",
    ]
):
    history_txt = Path(PATH).read_text(encoding="utf-8")

    history_data = np.array(
        [
            *map(
                lambda line: [*map(float, line.split(":"))],
                history_txt.splitlines(),
            )
        ]
    )

    (
        n_gas_history,
        n_crystal_history,
        concentration_history,
        delta_gibbs_history,
        energy_change_history,
        crystal_sx_history,
        crystal_sy_history,
        crystal_sz_history,
        mk_step_history,
    ) = history_data

    data[ID] = {
        "n_gas_history": n_gas_history,
        "n_crystal_history": n_crystal_history,
        "concentration_history": concentration_history,
        "delta_gibbs_history": delta_gibbs_history,
        "energy_change_history": energy_change_history,
        "crystal_sx_history": crystal_sx_history,
        "crystal_sy_history": crystal_sy_history,
        "crystal_sz_history": crystal_sz_history,
        "mk_step_history": mk_step_history,
    }

In [14]:
models = {
    model["name"]: ModelSpec(
        name=model["name"],
        t1=model["t1"],
        t2=model["t2"],
        func=model["func"],
        init_guess=model["init_guess"],
        bounds=model["bounds"],
    )
    for model_id, model in MODELS.items()
    if model_id in [1, 2]
}


def find_best_fits(
    x,
    y,
    methods=("lm", "trf", "dogbox"),
):
    results = []
    for method in methods:
        for spec_name, spec in models.items():
            try:
                popt, _ = curve_fit(
                    spec.func,
                    x,
                    y,
                    p0=spec.init_guess,
                    bounds=spec.bounds,
                    method=method,
                    maxfev=10_000_000,
                )
                y_pred = spec.func(x, *popt)
                r2 = calculate_r2(y, y_pred)
                results.append((method, spec.name, popt.tolist(), r2))
            except Exception:
                continue

    return results


# # func_m = lambda x1, x2: np.sqrt(np.sum(np.power(x1 - x2, 2)))
# # func_m = lambda x1, x2: np.sum((x1 - x2) ** 2)
# func_m = lambda x1, x2: 1 - calculate_r2(x1, x2)
# # func_m = lambda x1, x2: (1 - calculate_r2(x1, x2)) ** 2

# func1_base = lambda popt, x, y, func: func_m(y, func(x, *popt))


# def find_best_fits1(x, y):
#     methods = [
#         "Nelder-Mead",
#         "Powell",
#         "CG",
#         "BFGS",
#         "L-BFGS-B",
#         "TNC",
#         "COBYLA",
#         "SLSQP",
#         "trust-constr",
#     ]

#     results = []
#     for method in methods:
#         for spec_name, spec in models.items():
#             try:
#                 objective_function = partial(func1_base, x=x, y=y, func=spec.func)

#                 initial_guess = spec.init_guess

#                 # bounds = [(None, None) for _ in initial_guess]
#                 bounds = [*zip(*spec.bounds)]

#                 result = minimize(
#                     objective_function, initial_guess, bounds=bounds, method=method
#                 )
#                 # print(result)

#                 popt = result.x

#                 y_pred = spec.func(x, *popt)
#                 r2 = calculate_r2(y, y_pred)
#                 results.append((method, spec.name, popt.tolist(), r2))
#             except Exception:
#                 continue

#     return results

In [15]:
# X = np.copy(data[0]["mk_step_history"])
# Y = np.copy(data[0]["n_crystal_history"])
# models["exp001"].init_guess = [1150, -4e-05, 25800]
# models["exp002"].init_guess = [0.09, 6e-06, 1200, -4e-05, 25800]

# fits = find_best_fits(np.float64(X[:]), np.float64(Y[:]))
# fits = sorted(fits, key=lambda tup: tup[3], reverse=True)

# best = {}
# for method, model_name, params, r2 in fits:
#     # print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
#     if model_name not in best:
#         best[model_name] = {"m": models[model_name], "params": params, "r2": r2}
#         print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
# print()
# data[0]["fits"] = best


# X = np.copy(data[1]["mk_step_history"])
# Y = np.copy(data[1]["energy_change_history"])
# models["exp001"].init_guess = [1e-17, -2e-08, -1e-17]
# models["exp002"].init_guess = [1e-17, -6e-08, 5e-18, -5e-09, -1e-17]

# fits = find_best_fits(np.float64(X[:]), np.float64(Y[:]))
# fits = sorted(fits, key=lambda tup: tup[3], reverse=True)

# best = {}
# for method, model_name, params, r2 in fits:
#     # print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
#     if model_name not in best:
#         best[model_name] = {"m": models[model_name], "params": params, "r2": r2}
#         print(f"Model {model_name} via {method}: R² = {r2:.6f}, params = {params}")
# print()
# data[1]["fits"] = best

In [16]:
# exp001 = data[0]["fits"]["exp001"]
# exp002 = data[0]["fits"]["exp002"]
# print(exp001["m"].t2.format(*map(lambda v: f"{v:.5e}", exp001["params"])))
# # print(exp002["m"].t2.format(*map(lambda v: f"{v:.5e}", exp002["params"])))
# print()

# exp001 = data[0]["fits1"]["exp001"]
# exp002 = data[0]["fits1"]["exp002"]
# print(exp001["m"].t2.format(*map(lambda v: f"{v:.5e}", exp001["params"])))
# # print(exp002["m"].t2.format(*map(lambda v: f"{v:.5e}", exp002["params"])))

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

d = data[0]
X = np.copy(d["mk_step_history"])
sx, sy, sz = (d["crystal_sx_history"], d["crystal_sy_history"], d["crystal_sz_history"])
start_point = round(X.size * 0.50)
sx_mean, sy_mean, sz_mean = (
    round(sx[start_point:].mean()),
    round(sy[start_point:].mean()),
    round(sz[start_point:].mean()),
)
datasets = [
    (
        "Mean ΔX: {}".format(sx_mean),
        [X[start_point], X[-1]],
        [sx_mean, sx_mean],
        "solid",
        1.1,
        None,
        ".50",
    ),
    (
        "Mean ΔY: {}".format(sy_mean),
        [X[start_point], X[-1]],
        [sy_mean, sy_mean],
        "solid",
        1.1,
        None,
        ".25",
    ),
    (
        "Mean ΔZ: {}".format(sz_mean),
        [X[start_point], X[-1]],
        [sz_mean, sz_mean],
        "solid",
        1.1,
        None,
        ".75",
    ),
    ("ΔX", X, sx, "solid", 0.55, "s", ".50"),
    ("ΔY", X, sy, "solid", 0.55, "o", ".25"),
    ("ΔZ", X, sz, "solid", 0.55, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 10,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("$Δ Size$")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig("fig_9_4.png", format="png")
plt.show()
plt.close()

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

d = data[1]
X = np.copy(d["mk_step_history"])
sx, sy, sz = (d["crystal_sx_history"], d["crystal_sy_history"], d["crystal_sz_history"])
start_point = round(X.size * 0.50)
sx_mean, sy_mean, sz_mean = (
    round(sx[start_point:].mean()),
    round(sy[start_point:].mean()),
    round(sz[start_point:].mean()),
)
datasets = [
    (
        "Mean ΔX: {}".format(sx_mean),
        [X[start_point], X[-1]],
        [sx_mean, sx_mean],
        "solid",
        1.1,
        None,
        ".50",
    ),
    (
        "Mean ΔY: {}".format(sy_mean),
        [X[start_point], X[-1]],
        [sy_mean, sy_mean],
        "solid",
        1.1,
        None,
        ".25",
    ),
    (
        "Mean ΔZ: {}".format(sz_mean),
        [X[start_point], X[-1]],
        [sz_mean, sz_mean],
        "solid",
        1.1,
        None,
        ".75",
    ),
    ("ΔX", X, sx, "solid", 0.55, "s", ".50"),
    ("ΔY", X, sy, "solid", 0.55, "o", ".25"),
    ("ΔZ", X, sz, "solid", 0.55, "^", ".75"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 10,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("$Δ Size$")
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig("fig_10_4.png", format="png")
plt.show()
plt.close()